In [1]:
# Import necessary libraries
import os
from vllm import LLM, SamplingParams
from vllm.steer_vectors.request import SteerVectorRequest, VectorConfig
from transformers import AutoTokenizer

# Set environment variables
os.environ["CUDA_VISIBLE_DEVICES"] = "6"

# Initialize LLM with steering vector capability
llm = LLM(
    model="/data/zju-46/shenyl/hf/model/Qwen/Qwen2.5-1.5B-Instruct/",
    enable_steer_vector=True,
    enforce_eager=True,
    tensor_parallel_size=1
)

INFO 11-01 22:46:02 [utils.py:253] non-default args: {'disable_log_stats': True, 'enforce_eager': True, 'enable_steer_vector': True, 'model': '/data/zju-46/shenyl/hf/model/Qwen/Qwen2.5-1.5B-Instruct/'}
INFO 11-01 22:46:02 [model.py:657] Resolved architecture: Qwen2ForCausalLM
INFO 11-01 22:46:02 [model.py:1746] Using max model len 32768
INFO 11-01 22:46:04 [scheduler.py:211] Chunked prefill is enabled with max_num_batched_tokens=8192.
INFO 11-01 22:46:04 [vllm.py:414] Cudagraph is disabled under eager mode
(EngineCore_DP0 pid=1772444) INFO 11-01 22:46:05 [core.py:94] Initializing a V1 LLM engine (v0.1.dev10888+g9d4fd0da4.d20251031) with config: model='/data/zju-46/shenyl/hf/model/Qwen/Qwen2.5-1.5B-Instruct/', speculative_config=None, tokenizer='/data/zju-46/shenyl/hf/model/Qwen/Qwen2.5-1.5B-Instruct/', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=a

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


(EngineCore_DP0 pid=1772444) INFO 11-01 22:46:09 [default_loader.py:314] Loading weights took 1.00 seconds
(EngineCore_DP0 pid=1772444) INFO 11-01 22:46:09 [steer_vector_model_runner_mixin.py:36] Initialized SteerVector worker manager
(EngineCore_DP0 pid=1772444) INFO 11-01 22:46:09 [steer_vector_model_runner_mixin.py:50] Wrapping model with steer vector support
(EngineCore_DP0 pid=1772444) INFO 11-01 22:46:09 [hidden_states_model_runner_mixin.py:90] Wrapped 28 decoder layers for hidden states capture
(EngineCore_DP0 pid=1772444) INFO 11-01 22:46:10 [gpu_model_runner.py:2971] Model loading took 2.8876 GiB and 1.181162 seconds
(EngineCore_DP0 pid=1772444) INFO 11-01 22:46:11 [gpu_worker.py:343] Available KV cache memory: 38.33 GiB
(EngineCore_DP0 pid=1772444) INFO 11-01 22:46:12 [kv_cache_utils.py:1247] GPU KV cache size: 1,435,360 tokens
(EngineCore_DP0 pid=1772444) INFO 11-01 22:46:12 [kv_cache_utils.py:1252] Maximum concurrency for 32,768 tokens per request: 43.80x
(EngineCore_DP0 pi

In [2]:
prompt_no_input_template = "<|im_start|>user\n%s<|im_end|>\n<|im_start|>assistant\n"
instruction = "Who are you?"
instruction2 = "Who am i?"
prompt = [prompt_no_input_template % instruction, prompt_no_input_template % instruction2]
# Generate baseline response without steering
example_answers = llm.generate(
    prompt,
    SamplingParams(
        temperature=0,
        max_tokens=16,
        skip_special_tokens=False,
    ),
)

# Display baseline response
print("=====Baseline=====")
print(example_answers[0].outputs[0].text)
print(example_answers[1].outputs[0].text)
# Configure steering vector request for emoji_loreft

sv_request = SteerVectorRequest(
    "emoji_loreft",
    1,
    steer_vector_local_path="./weight/",
    scale=1.0,
    target_layers=[8],
    algorithm="loreft",
    prefill_trigger_positions=[-1],
    # debug=True
)
# Generate response with emoji_loreft
output = llm.generate(
    prompt, 
    SamplingParams(
        temperature=0,
        max_tokens=16,
        skip_special_tokens=False,
    ), 
    steer_vector_request=sv_request
)

# Display emoji_loreft response
print("=====Emoji_loreft Steered=====")
print(output[0].outputs[0].text)
print(output[1].outputs[0].text)

Adding requests:   0%|          | 0/2 [00:00<?, ?it/s]

Processed prompts:   0%| | 0/2 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, 

=====Baseline=====
I am an artificial intelligence designed to assist with various tasks and answer questions to the
You are a large language model.


Adding requests:   0%|          | 0/2 [00:00<?, ?it/s]

Processed prompts:   0%| | 0/2 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, 

=====Emoji_loreft Steered=====
🤖💬🌐🧠
👤❓🔍🌟
